# Extract contours from GeoJSON

In [ ]:
jsonfile = "./Inputs/coastal_polygons.geojson"

In [ ]:
import json
import ogr
import osr
import datacube
import matplotlib.pyplot as plt

from dea_datahandling import load_ard
from dea_datahandling import mostcommon_crs
from dea_bandindices import calculate_indices
from dea_coastaltools import tidal_tag
from dea_spatialtools import contour_extract
from dea_plotting import display_map
from dea_plotting import rgb
from dea_plotting import map_shapefile
import xarray as xr
from scipy import stats
import pandas as pd
import numpy as np

import fiona
from shapely.geometry import mapping
from shapely.ops import nearest_points
from shapely.geometry import Point, LineString, MultiPoint
import geopandas as gpd

dc = datacube.Datacube(app='erosion_contours')

In [ ]:
def change_regress(row, x_vals, x_labels, std_dev=3):
    
    # Extract x (time) and y (distance) values
    x = x_vals
    y = row.values[1:].astype(np.float)
    
    # Drop NAN rows
    xy_df = np.vstack([x, y]).T
    is_valid = ~np.isnan(xy_df).any(axis=1)
    xy_df = xy_df[is_valid]
    valid_labels = x_labels[is_valid]
    
    # Remove outliers
    outlier_bool = (np.abs(stats.zscore(xy_df)) < float(std_dev)).all(axis=1)
    xy_df = xy_df[outlier_bool]
        
    # Compute linear regression
    lin_reg = stats.linregress(x=xy_df[:,0], 
                               y=xy_df[:,1])
    
    # Return slope, p-values and list of outlier years excluded from regression   
    return pd.Series({'slope': np.round(lin_reg.slope, 2), 
                      'pvalue': np.round(lin_reg.pvalue, 3),
                      'outliers': str(valid_labels[~outlier_bool]).replace('[', '').replace(']', '')})

In [ ]:
def get_contours(feature):
    
    # Extract polygon to determine load range for data
    polygon = ogr.CreateGeometryFromJson(str(feature['geometry']))

    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)

    target = osr.SpatialReference()
    target.ImportFromEPSG(3577)

    transform = osr.CoordinateTransformation(source, target)
    polygon.Transform(transform)
    
    poly_envelope = polygon.GetEnvelope()
    minX, maxX, minY, maxY = poly_envelope
    
    # Set parameters for dc load
    x_range = (minX, maxX)
    y_range = (minY, maxY)
    time_range = ('2000', '2018')
    crs = "EPSG:3577"
    res = (-30, 30)
    tide_range = (0.50, 1.00)

    products = ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    measurements = ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_swir_1']
    
    # Construct query
    query = {
    'y': y_range,
    'x': x_range,
    'time': time_range,
    'measurements': measurements,
    'crs': crs,
    'output_crs': crs,
    'resolution': res
    }

    landsat_ds = load_ard(
        dc=dc,
        products=products,
        group_by='solar_day',
        **query
    )
    
    # Calculate MNDWI
    landsat_ds = calculate_indices(landsat_ds, index='MNDWI', 
                               collection='ga_ls_3')
    
    # Calculate tides for each timestep in the satellite dataset
    landsat_ds = tidal_tag(ds=landsat_ds, tidepost_lat=None, tidepost_lon=None)
    
    # Calculate the min and max tide heights to include based on the % range
    min_tide, max_tide = landsat_ds.tide_height.quantile(tide_range)
    
    # Keep timesteps larger than the min tide, and smaller than the max tide
    landsat_filtered = landsat_ds.sel(time=(landsat_ds.tide_height > min_tide) &
                                           (landsat_ds.tide_height <= max_tide))
    
    time_step = '1Y'

    # Combine into summary images by `time_step`
    landsat_summaries = (landsat_filtered.MNDWI
                         .compute()
                         .resample(time=time_step, closed='left')
                         .median('time'))
    
    # Set up attributes to assign to each waterline
    attribute_data = {'time': [str(i)[0:10] for i in landsat_summaries.time.values]}
    attribute_dtypes = {'time': 'str'}

    # Extract waterline contours for the '0' water index threshold:
    contour_gdf = contour_extract(
        z_values=[0],
        ds_array=landsat_summaries,
        ds_crs=landsat_ds.crs,
        ds_affine=landsat_ds.geobox.transform,
        output_shp=f'Outputs/contours.shp',
        attribute_data=attribute_data,
        attribute_dtypes=attribute_dtypes,
        min_vertices=50)
    
    contour_gdf.to_crs(epsg=4326)
    
    return(contour_gdf, landsat_ds, landsat_summaries)

In [ ]:
def get_rateofchange(contour_gdf, landsat_ds, landsat_summaries):
    # Set annual shoreline to use as a baseline
    baseline_year = contour_gdf.index[0]
    baseline_contour = contour_gdf.loc[baseline_year].geometry

    # Set up output shapefile
    schema = {'geometry': 'Point','properties': {'id': 'int'}}
    baseline_points_shp = f'Outputs/test_statistics.shp' 

    with fiona.open(baseline_points_shp, 'w', 'ESRI Shapefile', schema, crs=contour_gdf.crs) as output:

        # create points every 100 meters along the line
        for i, distance in enumerate(range(0, int(baseline_contour.length), 100)):
             point = baseline_contour.interpolate(distance)   
             output.write({'geometry': mapping(point), 'properties': {'id': i}})
            
    # Read points in as geopandas
    points_gdf = gpd.read_file(baseline_points_shp)

    # Copy geometry to baseline point
    points_gdf['p_baseline'] = points_gdf.geometry
    baseline_x_vals = points_gdf.geometry.x
    baseline_y_vals = points_gdf.geometry.y

    # Get array of water index values for baseline time period 
    baseline_array = landsat_summaries.isel(time = baseline_year)
    
    # Iterate through all comparison years in contour gdf
    for comp_year in contour_gdf.index.unique().values:

        # Set comparison contour
        comp_contour = contour_gdf.loc[comp_year].geometry

        # Find nearest point on comparison contour
        points_gdf[f'p_{comp_year}'] = points_gdf.apply(lambda x: 
                                                        nearest_points(x.p_baseline, comp_contour)[1], axis=1)

        # Compute distance between baseline and comparison year points
        points_gdf[f'{comp_year}'] = points_gdf.apply(lambda x: x.geometry.distance(x[f'p_{comp_year}']), axis=1)

        # Extract comparison array
        comp_array = landsat_summaries.isel(time = comp_year)

        # Convert baseline and comparison year points to geoseries to allow easy access to x and y coords
        comp_x_vals = gpd.GeoSeries(points_gdf[f'p_{comp_year}']).x
        comp_y_vals = gpd.GeoSeries(points_gdf[f'p_{comp_year}']).y

        # Sample NDWI values from arrays based on baseline and comparison points
        baseline_x_vals = xr.DataArray(baseline_x_vals, dims='z')
        baseline_y_vals = xr.DataArray(baseline_y_vals, dims='z')
        comp_x_vals = xr.DataArray(comp_x_vals, dims='z')
        comp_y_vals = xr.DataArray(comp_y_vals, dims='z')   
        points_gdf['index_comp_p1'] = comp_array.interp(x=baseline_x_vals, y=baseline_y_vals)
        points_gdf['index_baseline_p2'] = baseline_array.interp(x=comp_x_vals, y=comp_y_vals)

        # Compute directionality of change (negative = erosion, positive = accretion)    
        points_gdf['loss_gain'] = (points_gdf.index_baseline_p2 > points_gdf.index_comp_p1).astype(int).replace(to_replace=0, value=-1)
        points_gdf[f'{comp_year}'] = points_gdf[f'{comp_year}'] * points_gdf.loss_gain

    # Get list of cols to keep
    cols_to_keep = ['geometry'] + [str(val) for val in contour_gdf.index.unique().values]

    # Keep required columns
    points_gdf = points_gdf[cols_to_keep]
    points_gdf = points_gdf.round(2)
    
    x_years = np.array([int(i[:4]) for i in points_gdf.columns[1:]])
    
    # Compute change rates
    rate_out = points_gdf.apply(lambda x: change_regress(x, x_vals = x_years, x_labels = x_years, std_dev=3), axis=1)

    points_gdf[['mov_rate', 'mov_sig', 'mov_outl']] = rate_out
    
    # Set CRS
    points_gdf.crs = str(landsat_ds.crs)

    # Sort by descending absolute value and export
    points_gdf.reindex(points_gdf.mov_rate.abs().sort_values().index).to_file(baseline_points_shp)
    
    points_gdf = points_gdf[['geometry', 'mov_rate']]
    points_gdf.crs = {'init' :'epsg:3577'}
    points_gdf = points_gdf.to_crs({'init': 'epsg:4326'})
    
    return(points_gdf)

    #points_gdf.to_file('Outputs/contours_rateofchange.geojson', driver='GeoJSON')

In [ ]:
with open(jsonfile) as f:
    data = json.load(f)

for i, feature in enumerate(data['features']):
    contour, ds, ds_summary = get_contours(feature)
    contour.to_file(f'Outputs/contours_{i}.geojson', driver='GeoJSON')
    
    points = get_rateofchange(contour, ds, ds_summary)
    rate_of_change_file = f'Outputs/contours_rateofchange_{i}.geojson'
    points.to_file(rate_of_change_file, driver='GeoJSON')
    
    properties_dict = {'mov_rate': f'{np.mean(abs(points.mov_rate)).round(2)}',
                  'mov_points_file': 'contours_rateofchange.geojson'}

    data['features'][i]['properties'] = properties_dict

with open('Outputs/updated_polygons.geojson', 'w') as outfile:
    json.dump(data, outfile)